In [46]:
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Embedding
from keras.callbacks import LambdaCallback

In [47]:
# load doc into memory
with open('sequences.txt', encoding='utf-8') as f:
	texts = f.read().split('\n')

In [110]:
# Load tokenizer and convert word sequences to their integers
tokenizer = pickle.load(open('tokenizer.pkl', 'rb'))
sequences = np.asarray(tokenizer.texts_to_sequences(texts))
vocab_size = len(tokenizer.word_index) + 1
del tokenizer
print(sequences[:5])

[list([4, 494, 495]) list([494, 495, 1]) list([495, 1, 496])
 list([1, 496, 252]) list([496, 252, 497])]


In [92]:
# Embeddings
embeddings = pickle.load(open('embeddings.pkl', 'rb'))
embedding_dim = embeddings.shape[1]
print('embeddings shape: {}'.format(embeddings.shape))

embeddings shape: (1292, 50)


In [96]:
# separate into input and output
X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

ValueError: setting an array element with a sequence.

In [98]:
y

array([list([4, 494]), list([494, 495]), list([495, 1]), ...,
       list([1291, 1]), list([1, 493]), list([493, 7])], dtype=object)

In [89]:
# separate into input and output
# X is a sequence of integers
# X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)
# seq_length = X.shape[1]

ValueError: setting an array element with a sequence.

In [43]:
# define model
model = Sequential([
    Embedding(vocab_size, embedding_dim, weights=[embeddings], input_length=seq_length, trainable=False),
    GRU(100, return_sequences=True),
    GRU(100),
    Dense(100, activation='relu'),
    Dense(vocab_size, activation='softmax'),
])
model.summary()
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# callback
def on_epoch_end(epoch, logs):
    # save the model to a file
    model.save('model.h5')

callbacks = LambdaCallback(on_epoch_end=on_epoch_end)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 2, 50)             75650     
_________________________________________________________________
gru_9 (GRU)                  (None, 2, 100)            45300     
_________________________________________________________________
gru_10 (GRU)                 (None, 100)               60300     
_________________________________________________________________
dense_9 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_10 (Dense)             (None, 1513)              152813    
Total params: 344,163
Trainable params: 268,513
Non-trainable params: 75,650
_________________________________________________________________


In [44]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [45]:
# fit model
model.fit(X, y, batch_size=10, epochs=100, callbacks=[callbacks])

Epoch 1/100
9992/9992 [==============================] - 10s 1ms/step - loss: 5.7310 - acc: 0.0704
Epoch 2/100
9992/9992 [==============================] - 9s 877us/step - loss: 5.2435 - acc: 0.0942
Epoch 3/100
9992/9992 [==============================] - 9s 895us/step - loss: 5.0204 - acc: 0.1086
Epoch 4/100
9992/9992 [==============================] - 9s 869us/step - loss: 4.8295 - acc: 0.1257
Epoch 5/100
9992/9992 [==============================] - 9s 865us/step - loss: 4.6755 - acc: 0.1345
Epoch 6/100
9992/9992 [==============================] - 9s 912us/step - loss: 4.5388 - acc: 0.1435
Epoch 7/100
9992/9992 [==============================] - 9s 880us/step - loss: 4.4036 - acc: 0.1482
Epoch 8/100
9992/9992 [==============================] - 9s 867us/step - loss: 4.2636 - acc: 0.1512
Epoch 9/100
9992/9992 [==============================] - 9s 864us/step - loss: 4.1268 - acc: 0.1600
Epoch 10/100
9992/9992 [==============================] - 9s 882us/step - loss: 3.9881 - acc: 0.1678


Epoch 82/100
9992/9992 [==============================] - 9s 856us/step - loss: 1.0978 - acc: 0.6306
Epoch 83/100
9992/9992 [==============================] - 9s 857us/step - loss: 1.1021 - acc: 0.6314
Epoch 84/100
9992/9992 [==============================] - 9s 861us/step - loss: 1.0989 - acc: 0.6285
Epoch 85/100
9992/9992 [==============================] - 9s 859us/step - loss: 1.0855 - acc: 0.6309
Epoch 86/100
9992/9992 [==============================] - 9s 859us/step - loss: 1.0882 - acc: 0.6285
Epoch 87/100
9992/9992 [==============================] - 9s 861us/step - loss: 1.0759 - acc: 0.6300
Epoch 88/100
9992/9992 [==============================] - 9s 867us/step - loss: 1.0812 - acc: 0.6309
Epoch 89/100
9992/9992 [==============================] - 9s 861us/step - loss: 1.0716 - acc: 0.6297
Epoch 90/100
9992/9992 [==============================] - 9s 860us/step - loss: 1.0683 - acc: 0.6310 1s -
Epoch 91/100
9992/9992 [==============================] - 9s 862us/step - loss: 1.0682